In [19]:
import requests
from bs4 import BeautifulSoup
import tarfile
import shutil
 

# Login credentials
username = 'i.arisi@ebri.it' #replace with your own username and password 
password = '????????'

# Session object for persistent connection
session = requests.Session()

# Login URL of loni ppmi data 
login_url = 'https://ida.loni.usc.edu/web/ppmi-rnaseq-app-login.jsp' 
login_data = {
    'username': username,
    'password': password
}

# Send a POST request to login
response = session.post(login_url, data=login_data)
# Local path to save the downloaded file
local_path = '/Users/Utente/Desktop/Data_PPMI/download/PPMI_RNAseq_IR3_Release_new.pdf'  # Replace with the desired local path


# Check if login was successful
if response.status_code == 200:
    print('Login successful')
    # Continue with downloading data or performing other operations

    # Example: Download data from a specific URL
    data_url = 'https://ida.loni.usc.edu/download/files/genetic/2c6c21ac-1433-4871-972c-6e1751ce353e/ppmi/PPMI_RNAseq_IR3_Release_20210402.pdf'  # Replace with the actual data URL
    response = session.get(data_url)
    print('Data response PPMI_RNAseq_IR3_Release_20210402.pdf was successfull')
    
    response = requests.get(data_url, stream=True)

    # Save the downloaded file locally
    with open(local_path, 'wb') as f:
        response.raw.decode_content = True
        shutil.copyfileobj(response.raw, f)
    print('File downloaded successfully')

else:
    print('Login failed')

Login successful
Data response PPMI_RNAseq_IR3_Release_20210402.pdf was successfull
File downloaded successfully


In [10]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance

# Load the Iris dataset
data = load_iris()
X, y = data.data, data.target

# Set the number of repetitions for Random Forest training
num_repetitions = 1

# Set the number of folds for repeated stratified 10-fold cross-validation
num_folds = 2

# Set the random seed for reproducibility
random_seed = 42

# Define the function to train a Random Forest model and compute permutation feature importances
def train_rf_with_permutation_importance(X_train, y_train, X_test, y_test, num_features):
    rf_model = RandomForestClassifier(n_estimators=100, random_state=random_seed)
    rf_model.fit(X_train, y_train)

    perm_importance = permutation_importance(rf_model, X_test, y_test, n_repeats=3, random_state=random_seed)
    feature_importance = perm_importance.importances_mean

    # Get the indices of the top features based on their importance
    top_feature_indices = np.argsort(feature_importance)[-int(np.sqrt(num_features)):]
    
    return top_feature_indices

# Initialize lists to store the selected top features across all splits
selected_top_features = []

# Create the repeated stratified k-fold cross-validator
cv = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=random_seed)

# Repeat the process for a total of 'num_repetitions' times
for repetition in range(num_repetitions):
    print(f"Repetition {repetition+1}/{num_repetitions}")

    # Perform repeated stratified k-fold cross-validation
    for train_idx, test_idx in cv.split(X, y):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # Get the selected top features for this split
        top_features_split = train_rf_with_permutation_importance(X_train, y_train, X_test, y_test, len(X[0]))

        # Add the selected top features to the list
        selected_top_features.extend(top_features_split)

# Count the occurrences of each feature in the selected_top_features list
feature_counts = pd.Series(selected_top_features).value_counts()

# Get the indices of the top features based on their overall occurrence
final_top_feature_indices = feature_counts.index[:int(np.sqrt(len(X[0])))]

# Convert the Index to a list of integer indices
final_top_feature_indices = final_top_feature_indices.tolist()

# Get the names of the final selected top features
final_top_features = data.feature_names[final_top_feature_indices]

print("Final selected top features:")
print(final_top_features)


Repetition 1/1


TypeError: list indices must be integers or slices, not list